In [55]:
from tqdm import tqdm
import pandas as pd
import constants as c
from importlib import reload

reload(c)

<module 'constants' from 'g:\\Shared drives\\Data\\Analysis\\Geography\\src\\constants.py'>

- Cleaning column and row names

In [56]:
df = pd.read_excel(c.raw_data_file_lprw, **c.read_options_lprw)
#def clean_LGA_name(LGA):
#    return(LGA.split(' (')[0])
def clean_postcode_name(postcode):
    return(postcode.split(', ')[0].split(' ')[0])
#df.rename(clean_LGA_name, axis='columns', inplace=True)
df.rename(clean_postcode_name, axis='index', inplace=True)
df = df[df.columns[1:-1]] #Removes 'LGA' from columns


In [57]:
df_lga_2016_to_2021 = pd.read_csv(c.geography_transforms['LGA']['2016-2021'])
df_lga_2016_to_2021 = df_lga_2016_to_2021.sort_values(by='RATIO_FROM_TO', ascending=False)
df_lga_2016_to_2021 = df_lga_2016_to_2021.drop_duplicates(subset='LGA_NAME_2016', keep='first')

In [58]:
df_poa_2016_to_2021 = pd.read_csv(c.geography_transforms['POA']['2016-2021'])
df_poa_2016_to_2021 = df_poa_2016_to_2021.sort_values(by='RATIO_FROM_TO', ascending=False)
df_poa_2016_to_2021 = df_poa_2016_to_2021.drop_duplicates(subset='POA_NAME_2016', keep='first')

In [59]:
postcodes, lgas, households = [], [], []

for col in df.columns:
    subset = df[df[col] > 0][col]
    postcodes += list(subset.index)
    households += list(subset)
    lgas += list([col])*len(subset)

postcode_totals = df.sum(axis='columns')
postcode_totals = postcode_totals.reset_index(name='Postcode Totals').rename({'index': 'Postcode'}, axis='columns')

lga_totals = df.sum(axis='rows')
lga_totals = lga_totals.reset_index(name='LGA Totals').rename({'index': 'LGA'}, axis='columns')

df_new = pd.DataFrame({'Postcode': postcodes, 'LGA': lgas, 'Rented Households': households})

df_new = df_new.merge(postcode_totals, on='Postcode', how='left')
df_new = df_new.merge(lga_totals, on='LGA', how='left')
df_new.sort_values(by=['Postcode'])

df_new['Postcode Fraction'] = df_new['Rented Households']/df_new['Postcode Totals']
df_new['LGA Fraction'] = df_new['Rented Households']/df_new['LGA Totals']

for col in ['Rented Households', 'Postcode Totals', 'LGA Totals']:
    df_new[col] = df_new[col].astype(int)

In [60]:
df_new = df_new.merge(df_lga_2016_to_2021, left_on='LGA', right_on='LGA_NAME_2016', how='left')
cols = ['Postcode', 'LGA_NAME_2021', 'Rented Households', 'Postcode Totals', 'LGA Totals',
       'Postcode Fraction', 'LGA Fraction', 'LGA_NAME_2016', 'RATIO_FROM_TO']
df_new = df_new[cols].rename({'LGA_NAME_2021': 'LGA', 'RATIO_FROM_TO': 'LGA_RATIO_FROM_TO'}, axis='columns')

df_new = df_new.merge(df_poa_2016_to_2021, left_on='Postcode', right_on='POA_NAME_2016', how='left')
cols = ['POA_NAME_2021', 'LGA', 'Rented Households', 'Postcode Totals', 'LGA Totals',
       'Postcode Fraction', 'LGA Fraction', 'LGA_NAME_2016', 'LGA_RATIO_FROM_TO', 'POA_NAME_2016', 'RATIO_FROM_TO'] 
df_new = df_new[cols].rename({'POA_NAME_2021': 'Postcode', 'RATIO_FROM_TO': 'POA_RATIO_FROM_TO'}, axis='columns')


In [61]:
print("Postcodes:", len(df_new['Postcode'].drop_duplicates()))
print("LGAs:", len(df_new['LGA'].drop_duplicates()))
df_new

Postcodes: 608
LGAs: 129


,Postcode,LGA,Rented Households,Postcode Totals,LGA Totals,Postcode Fraction,LGA Fraction,LGA_NAME_2016,LGA_RATIO_FROM_TO,POA_NAME_2016,POA_RATIO_FROM_TO
0,2640,Albury,4271,4284,6557,0.996965,0.651365,Albury (C),1.000000,2640,1.0
1,2641,Albury,2286,2286,6557,1.000000,0.348635,Albury (C),1.000000,2641,1.0
2,2350,Armidale Regional,3215,3246,3591,0.990450,0.895294,Armidale Regional (A),0.970337,2350,1.0
3,2359,Armidale Regional,4,52,3591,0.076923,0.001114,Armidale Regional (A),0.970337,2359,1.0
4,2360,Armidale Regional,6,1634,3591,0.003672,0.001671,Armidale Regional (A),0.970337,2360,1.0
...,...,...,...,...,...,...,...,...,...,...,...
854,2584,Yass Valley,41,41,927,1.000000,0.044229,Yass Valley (A),1.000000,2584,1.0
855,2621,Yass Valley,3,283,927,0.010601,0.003236,Yass Valley (A),1.000000,2621,1.0
856,2648,Unincorporated NSW,3,206,77,0.014563,0.038961,Unincorporated NSW,1.000000,2648,1.0
857,2880,Unincorporated NSW,36,1694,77,0.021251,0.467532,Unincorporated NSW,1.000000,2880,1.0


In [62]:
df_new.sort_values(by='Postcode').to_csv(c.processed_data_dir + 'LGA To Postcode 2016 Rented Household Weightings.csv', index=False)